In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graduate-admissions/Admission_Predict.csv
/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv


In [2]:
df=pd.read_csv("/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [3]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [4]:
x=df.drop(['Serial No.','Chance of Admit '],axis=1)
y=df['Chance of Admit ']

In [5]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=1)

In [6]:
xtrain.shape

(400, 7)

In [7]:
xtest.shape

(100, 7)

In [8]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
xtrain_sc=sc.fit_transform(xtrain)
xtest_sc=sc.transform(xtest)

In [9]:
xtrain.shape

(400, 7)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [11]:
from keras.callbacks import EarlyStopping

In [63]:
callback=EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=10,
    verbose=-1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,

)

In [72]:
model=Sequential()
model.add(Dense(7,activation='relu',input_dim=7,kernel_regularizer=tensorflow.keras.regularizers.l2(0.01)))
#model.add(Dropout(0.2))
model.add(Dense(7,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='linear'))

In [73]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 7)                 56        
                                                                 
 dense_31 (Dense)            (None, 7)                 56        
                                                                 
 dropout_10 (Dropout)        (None, 7)                 0         
                                                                 
 dense_32 (Dense)            (None, 1)                 8         
                                                                 
Total params: 120
Trainable params: 120
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [75]:
model.fit(xtrain_sc,ytrain,epochs=1000,validation_split=0.2,callbacks=callback)

Epoch 1/1000
10/10 [==============================] - 1s 19ms/step - loss: 0.3047 - val_loss: 0.2517
Epoch 2/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.2385 - val_loss: 0.1832
Epoch 3/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1950 - val_loss: 0.1390
Epoch 4/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1755 - val_loss: 0.1151
Epoch 5/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1545 - val_loss: 0.1043
Epoch 6/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1426 - val_loss: 0.0987
Epoch 7/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1351 - val_loss: 0.0950
Epoch 8/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1282 - val_loss: 0.0903
Epoch 9/1000
10/10 [==============================] - 0s 5ms/step - loss: 0.1255 - val_loss: 0.0849
Epoch 10/1000
10/10 [==============================] - 0s 4ms/step - loss: 0.1181 - val_loss: 0.080

In [76]:
ypred=model.predict(xtest_sc)

4/4 [==============================] - 0s 2ms/step


In [77]:
from sklearn.metrics import r2_score
r2_score(ytest,ypred)

0.7363630102798602